In [1]:
from bs4 import BeautifulSoup
import csv
import os
import time
import json
import requests
import re
import pandas as pd
import numpy as np
import lxml

In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
headers = {
    'accept': '*/*',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}


In [4]:
list_spec = [
    'year',
    'status',
    'dimensions',
    'weight',
    'sim',
    'displaytype',
    'displaysize',
    'displayresolution',
    'os',
    'chipset',
    'cpu',
    'gpu',
    'memoryslot',
    'internalmemory',
    'cam1modules',
    'cam1features',
    'cam1video',
    'cam2modules',
    'cam2video',
    'wlan',
    'bluetooth',
    'gps',
    'nfc',
    'radio',
    'batdescription1',
    'price'
]

In [5]:
#function for downoload page of brand/telephones list/telephone spec

def get_soup_page(url_add):
    try:
        time.sleep(30)
        url = 'https://www.gsmarena.com/' + url_add
        req = requests.get(url, headers = headers)
        soup = BeautifulSoup(req.text, 'lxml')
        return soup
    except:
        time.sleep(300)
        url = 'https://www.gsmarena.com/' + url_add
        req = requests.get(url, headers = headers)
        soup = BeautifulSoup(req.text, 'lxml')
        return soup

In [6]:
#get all info about mobile brands

def get_links_brands():
    url_add = 'makers.php3'
    soup_brands = get_soup_page(url_add)
    links_brands = soup_brands.find(class_= 'st-text').find_all('a', href =  True)
    return links_brands

In [7]:
#get list of all mobile phones by one brand

def get_links_phone_brand(link_brand):
    url_add = link_brand.get('href')
    soup_phones = get_soup_page(url_add)
    links_phones = soup_phones.find(class_= 'makers').find_all('a', href =  True)
    try:
        pages_phones = soup_phones.find(class_= 'nav-pages').find_all('a', href =  True)
        links_page_phones = get_links_phone_from_pages(pages_phones)
        links_phones += links_page_phones
    except AttributeError:
        pass
    return links_phones

In [8]:
#get links of mobile phones on different pages by one brand

def get_links_phone_from_pages(pages_phones):
    links_page_phones = []
    for page in pages_phones:
        url_add = page.get('href')
        soup_phones_page = get_soup_page(url_add)
        var_links_phones = soup_phones_page.find(class_= 'makers').find_all('a', href =  True)
        links_page_phones += var_links_phones
    return links_page_phones

In [9]:
#get specification of mobile phone with "list_spec"

def get_spec_phone(link_phone):
    url_add = link_phone.get('href')
    soup_phone = get_soup_page(url_add)
    phone_dict = {}
    try:
        phone_title = soup_phone.find(class_='specs-phone-name-title').text
    except:
        print('No title')
    phone_dict['title'] = phone_title
    for spec in list_spec:
        try:
            var = soup_phone.find('td', {'data-spec': spec}).text
            phone_dict[spec] = var
        except AttributeError:
            phone_dict[spec] = 'no data'
    return phone_dict

In [10]:
#main funcition for creating mobile phone dataset

def get_phone_dataframe(list_spec):
    try:
        links_brands = get_links_brands()
        df_all_phones = pd.DataFrame(columns = list_spec)
        for link_brand in links_brands:
            links_phones = get_links_phone_brand(link_brand)
            for link_phone in links_phones:
                phone_dict = get_spec_phone(link_phone)
                phone_dict['brand'] = link_brand.get('href').split('-')[0]
                print(phone_dict['title'])
                df_all_phones = df_all_phones.append(phone_dict, ignore_index = True)
        return df_all_phones
    except KeyboardInterrupt:
        return df_all_phones

In [ ]:
df_export_phones = get_phone_dataframe(list_spec)


In [ ]:
# save dataset to excel file

df_export_phones.to_excel('mobile_phone_dataset.xlsx', index = False)